In [1]:
import logging
import sys
import pandas as pd

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [2]:
from llama_index.core.evaluation import DatasetGenerator, RelevancyEvaluator
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Response
from llama_index.llms.groq import Groq
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.tools import FunctionTool
from semantic_router import Route
from semantic_router.encoders import HuggingFaceEncoder
from semantic_router.layer import RouteLayer
from IPython.display import Markdown
from ema_workbench import load_results
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn import metrics
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import seaborn as sns
from llama_index.experimental.query_engine import PandasQueryEngine
from llama_index.core.schema import Document

/home/ana/Documents/Proyectos/microminer-eval/.venv/lib/python3.11/site-packages/ema_workbench/em_framework/__init__.py:101: UserWarning: ipyparallel not installed - IpyparalleEvaluator not available
  warnings.warn("ipyparallel not installed - IpyparalleEvaluator not available")


In [3]:
import nest_asyncio

nest_asyncio.apply()

In [4]:
Settings.llm = Groq(model="llama3-8b-8192", api_key="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", base_url="https://api.groq.com/openai/v1")
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


In [5]:
def run_agglomerative(df, k, threshold=200, n_pca=None, normalize=False, show_dendogram=False, archstructure=None):    
      if normalize:
        sample = StandardScaler().fit_transform(df)
      else:
        sample = df.values
    
      if n_pca is not None:
        pca = PCA(n_components=n_pca)
        sample_pca = sample
        model = AgglomerativeClustering(n_clusters=k, affinity='euclidean', linkage='ward',
                                        connectivity=archstructure, distance_threshold=threshold)
        model.fit(sample_pca)
        X = sample_pca
      else:
        model = AgglomerativeClustering(n_clusters=k, affinity='precomputed', linkage='single',
                                      connectivity=archstructure, distance_threshold=threshold)
        model.fit(sample)
      labels = model.labels_
      fixed_labels = np.where(model.labels_ < 0, 0, model.labels_)
      classes = set(fixed_labels)
      if len(classes) > 1:
        if n_pca is not None:
          silhouette = metrics.silhouette_score(sample_pca, fixed_labels)
        else:
          silhouette = metrics.silhouette_score(sample, fixed_labels)
      else:
        silhouette = 0.0    
      return fixed_labels, model, silhouette


def load_data(project_path, project_name):
    model_filename = f"{project_path}/{project_name}_128scenarios_nopolicies_sobol" #.tar.gz'
    experiments_df, outcomes = load_results(model_filename+ '.tar.gz')
    outcomes = pd.DataFrame(outcomes)
    experiments = experiments_df
    with open(model_filename+'_model.pkl', 'rb') as input:
        uncertainties_problem = pickle.load(input)
    with open(model_filename+'_partitions.pkl', 'rb') as input:
        partitions = pickle.load(input)
    similarity_filename = f"{project_path}/{project_name}_omega_scores.csv"
    partitions_distance = 1 - pd.read_csv(similarity_filename, index_col=0)
    stable_solutions_filename = f"{project_path}/{project_name}_stable_solutions.pkl"
    with open(stable_solutions_filename, 'rb') as f:
         stable_solutions = pickle.load(f)
         other_labels = stable_solutions.keys()
    mds = MDS(dissimilarity='precomputed', random_state=0)
    embeddings_2d_partitions = mds.fit_transform(partitions_distance)
    partition_labels_2d, _, silhouette = run_agglomerative(embeddings_2d_partitions, k=5, threshold=None, show_dendogram=True, normalize=True, n_pca=2)
    partition_labels = partition_labels_2d
    return outcomes

In [6]:
outcomes = load_data("../jpetstore", "jpetstore")

INFO:EMA.ema_workbench.util.utilities:results loaded successfully from /home/ana/Documents/Proyectos/microminer-eval/jpetstore/jpetstore_128scenarios_nopolicies_sobol.tar.gz
results loaded successfully from /home/ana/Documents/Proyectos/microminer-eval/jpetstore/jpetstore_128scenarios_nopolicies_sobol.tar.gz
results loaded successfully from /home/ana/Documents/Proyectos/microminer-eval/jpetstore/jpetstore_128scenarios_nopolicies_sobol.tar.gz


/home/ana/Documents/Proyectos/microminer-eval/.venv/lib/python3.11/site-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(
/home/ana/Documents/Proyectos/microminer-eval/.venv/lib/python3.11/site-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


In [12]:
outcomes.shape

(768, 5)

In [ ]:
def describe_column(df, row):
    data = ', '.join([f"{col}: {row[col]}" for col in df.columns])
    key = f"Decomposition id {row.index} with metrics: "
    return key + data

def df_to_documents(df):
    return [Document(text=describe_column(df, row), metadata={col: row[col] for col in df.columns}) for _, row in df.iterrows()]

# Convert DataFrame to Document list
documents = df_to_documents(outcomes)

In [ ]:
data_generator = DatasetGenerator.from_documents(documents[0:10])

In [ ]:
documents

In [36]:
documents

[Document(id_='00161b0f-1d19-4f00-b7ce-5ea8bd4bd3f0', embedding=None, metadata={'n_partitions': 5.0, 'modularity': 0.0059861900141409, 'ned': 0.25, 'density': 4.734020618556701, 'noise_classes': 17.0}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="Decomposition id Index(['n_partitions', 'modularity', 'ned', 'density', 'noise_classes'], dtype='object') with metrics: n_partitions: 5.0, modularity: 0.0059861900141409, ned: 0.25, density: 4.734020618556701, noise_classes: 17.0", mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='65c3170b-a01d-4070-aa2e-b9ceffb6f5eb', embedding=None, metadata={'n_partitions': 6.0, 'modularity': 0.0159809796290929, 'ned': 0.75, 'density': 5.400916380297825, 'noise_classes': 17.0}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="Decomposition id I

In [38]:
eval_questions = data_generator.generate_questions_from_nodes()

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST ht

/home/ana/Documents/Proyectos/microminer-eval/.venv/lib/python3.11/site-packages/llama_index/core/evaluation/dataset_generation.py:296: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return QueryResponseDataset(queries=queries, responses=responses_dict)


In [39]:
eval_questions

['Here are 10 questions based on the provided context information:',
 "What is the value of 'n_partitions' in the decomposition?",
 'a) 3.0',
 'b) 5.0',
 'c) 10.0',
 'd) 20.0',
 'Answer: b) 5.0',
 "What is the value of 'modularity' in the decomposition?",
 'a) 0.01',
 'b) 0.0059861900141409',
 'c) 0.1',
 'd) 0.5',
 'Answer: b) 0.0059861900141409',
 "What is the value of 'ned' in the decomposition?",
 'a) 0.1',
 'b) 0.25',
 'c) 0.5',
 'd) 0.75',
 'Answer: b) 0.25',
 "What is the value of 'density' in the decomposition?",
 'a) 1.0',
 'b) 4.734020618556701',
 'c) 10.0',
 'd) 20.0',
 'Answer: b) 4.734020618556701',
 "How many 'noise_classes' are present in the decomposition?",
 'a) 10.0',
 'b) 15.0',
 'c) 17.0',
 'd) 20.0',
 'Answer: c) 17.0',
 'What is the data type of the index in the decomposition?',
 'a) integer',
 'b) float',
 'c) object',
 'd) string',
 'Answer: c) object',
 'What is the name of the index in the decomposition?',
 'a) n_partitions',
 'b) modularity',
 'c) ned',
 'd) I

In [22]:
evaluator = RelevancyEvaluator()

In [23]:
vector_index = VectorStoreIndex.from_documents(documents)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
def display_eval_df(query: str, response: Response, eval_result: str) -> None:
    eval_df = pd.DataFrame(
        {
            "Query": [query],
            "Response": [str(response)],
            "Source": [(
                response.source_nodes[0].node.get_content()[:1000] + "..."
            )],
            "Evaluation Result": [eval_result.passing],
        },
        index=[0],
    )
    eval_df = eval_df.style.set_properties(
        **{
            "inline-size": "600px",
            "overflow-wrap": "break-word",
        },
        subset=["Response", "Source"]
    )
    display(eval_df)


In [25]:
query_engine = PandasQueryEngine(df=outcomes, verbose=True)
response_vector = query_engine.query(eval_questions[1])
eval_result = evaluator.evaluate_response(
    query=eval_questions[1], response=response_vector
)

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```
df['n_partitions']
```
> Pandas Output: 0      5.0
1      6.0
2      5.0
3      6.0
4      4.0
      ... 
763    7.0
764    4.0
765    4.0
766    7.0
767    4.0
Name: n_partitions, Length: 768, dtype: float64


In [26]:
display_eval_df(eval_questions[1], response_vector, eval_result)

IndexError: list index out of range